In [28]:
import pandas as pd
import os

In [29]:
importFile = 'CombinedTestingFrame_A2_A3_O3.txt'
outName = 'CombinedTestingFrame_A2_A3_O3_short.txt'
df_import = pd.read_csv(importFile,names=['ID','label','extra'],sep='\t')

In [30]:
df_import['label'].value_counts()

label
17    28025
8     12950
0     11700
11     8275
3      7425
2      7050
9      5400
15     4925
14     3925
13     2425
7      2400
6      2100
12     1750
5       700
16      200
Name: count, dtype: int64

In [31]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s',1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES','NIFTI')+'.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath':NiftiPath, 'slicenum':slicenum})

In [32]:
df_NS = df_import.apply(extractNiftiFilepathAndSlicenum,axis=1, result_type='expand')

In [33]:
df = df_import.merge(df_NS,left_index=True,right_index=True,how='inner')

In [34]:
vc = df['label'].value_counts()
vc

label
17    28025
8     12950
0     11700
11     8275
3      7425
2      7050
9      5400
15     4925
14     3925
13     2425
7      2400
6      2100
12     1750
5       700
16      200
Name: count, dtype: int64

In [35]:
min = vc.iloc[-2]
min

700

In [36]:
from sklearn.model_selection import  train_test_split

In [37]:
FinalFrame = pd.DataFrame(columns=['ID','label','extra'])
for l in df['label'].unique():
    temp = df[df['label']==l].groupby('NiftiPath')
    print(len(temp))
    try:
        keep, discard = train_test_split(list(range(len(temp))), train_size=int(min/25))
    except:
        keep = list(range(int(min/25)))
    for i,frame in enumerate(temp):
        if i in keep:
           FinalFrame =  pd.concat((FinalFrame,frame[1]),axis=0)

297
1121
197
468
518
97
157
84
8
28
282
216
331
70
96


In [38]:
FinalFrame['label'].value_counts()

label
3     700
17    700
15    700
0     700
8     700
13    700
14    700
6     700
5     700
2     700
9     700
11    700
12    700
7     700
16    200
Name: count, dtype: int64

In [39]:
FinalFrame = FinalFrame.reset_index(drop=True)

In [40]:
FinalFrame[['ID','label','extra']].to_csv(outName,header=False,index=False,sep='\t')

In [41]:
FinalFrame

,ID,label,extra,NiftiPath,slicenum
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3.0
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,5.0
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,7.0
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,16.0
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,20.0
...,...,...,...,...,...
9995,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,7,0,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,9.0
9996,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,7,0,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,10.0
9997,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,7,0,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,11.0
9998,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,7,0,/trinity/home/r098375/DDS/data/OASIS/OASIS3/te...,0.0
